In [ ]:
#from keras.engine.training import Model
#from keras.applications.resnet50 import ResNet50
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.preprocessing import image
#from keras.utils import np_utils
#from keras.callbacks import EarlyStopping
import numpy as np
import os
import sys
import tensorflow as tf
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# CONSTANTS
IMAGE_WIDTH = 96
IMAGE_HEIGHT = 96
IMAGE_NUMBER_CHANNELS = 3
NUM_CLASSES = 102

# Data preprocessing
X = []
Y = []

i = 0

for root, dirs, files in os.walk("./flower_data/train"):
    
    for filename in files:
        img = image.load_img(os.path.join(root, filename), target_size=(IMAGE_HEIGHT, IMAGE_WIDTH))
        img = image.img_to_array(img)
        #img = img.astype('float32')
        img = img / 255
        #print(img)
        #break
        #img = img.astype('float32')
        #img = img / 255
        #img = np.expand_dims(img, axis=0)
        #img /= 255.
        #img = preprocess_input(img)
        
        X.append(img)
        Y.append(i)
        
    i += 1

X = np.asarray(X)
Y = np.asarray(Y)
print(X.shape)
print(Y.shape)
print("Number of classes: ", NUM_CLASSES)
np.save("X", X)
np.save("Y", Y)

In [ ]:
# Data load and splitting
X = np.load("X.npy")
Y = np.load("Y.npy")

Y = np_utils.to_categorical(Y, NUM_CLASSES)

mask = np.random.rand(len(X)) < 0.9

X_trn, X_tst = X[mask], X[~mask]
Y_trn, Y_tst = Y[mask], Y[~mask]

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_trn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(i))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
#opt = tf.keras.optimizers.adam(lr=0.0001, decay=1e-6)

model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://10.240.1.2:8470'))
)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_trn, Y_trn,
              batch_size=64,
              epochs=150,
              validation_data=(X_tst, Y_tst),
              shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=6)])

In [ ]:
# ResNet 50 model
model = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False, classes=i, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_NUMBER_CHANNELS))

# Additional line added
model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, classes=i, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_NUMBER_CHANNELS))

# Freeze layers
for layer in model.layers:
    layer.trainable = False
    
# Modificaciones
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='sigmoid')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = tf.keras.layers.Dense(1024, activation='sigmoid')(x) #dense layer 2
x = tf.keras.layers.Dense(512, activation='sigmoid')(x) #dense layer 3
preds = tf.keras.layers.Dense(i, activation='softmax')(x) #final layer with softmax activation

model = tf.keras.Model(inputs=model.input, outputs=preds)

print(len(model.layers))

model = tf.contrib.tpu.keras_to_tpu_model(model,
	strategy=tf.contrib.tpu.TPUDistributionStrategy(
		tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://10.240.1.2:8470')
    )
)

#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
'''model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)'''

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#early_stopping = EarlyStopping(monitor="val_loss", patience=3)
model.fit(X_trn, Y_trn, batch_size=32, epochs=500, verbose=2, validation_data=(X_tst, Y_tst))


# Results
loss, acc = model.evaluate(X_tst, Y_tst, verbose=0)
print("Test score: {:.2f} accuracy: {:.2f}%".format(loss, acc*100))